In [49]:
from aicspylibczi import CziFile
import numpy as np
import skimage as sk
import pandas as pd
import os
from glob import glob
import napari
from cellpose import models, io, utils
#import pyclesperanto as cle


In [205]:
def read_and_prep_images(czifile):
    scenes = [czifile.read_image(S=s)[0] for s in range(czifile.size[2])]
    squeezed = [np.squeeze(i) for i in scenes]
    nuclei_channel = [squeezed[i][-1,...] for i in range(len(squeezed))]
    gauss_nuc = [sk.filters.gaussian(img, sigma=3) for img in nuclei_channel]
    return scenes, squeezed, gauss_nuc, nuclei_channel        


In [ ]:
def segment_and_measure_CP():


In [206]:
file_paths = sorted(glob("E:\Fondufe-Mittendorf_Lab\OIC-150_Images\*.czi"))
czi_files = list(map(CziFile,file_paths))
scenes, squeezed, gauss_nuc, nuclei_channel = read_and_prep_images(czifile=czi_files[0])

In [213]:
gauss_nuc[0].shape

(24, 1012, 1012)

In [189]:
file_paths = sorted(glob("E:\Fondufe-Mittendorf_Lab\OIC-150_Images\*.czi"))
czi_files = list(map(CziFile,file_paths))


In [192]:
czi_files[0].size[2]

3

In [196]:
img = [czi_files[0].read_image(S=i)[0] for i in range(czi_files[0].size[2])]
squeezed = [np.squeeze(i) for i in img]

In [198]:
squeezed[0].shape

(4, 24, 1012, 1012)

In [163]:
img = np.squeeze(img)
print(img.shape)
split = np.split(img, img.shape[1], axis=1)

(3, 4, 27, 1012, 1012)


In [159]:
squeeze = np.squeeze(split)
squeeze[0].shape

(3, 27, 1012, 1012)

In [144]:
gauss = sk.filters.gaussian(test[-1,...],sigma=3)

In [147]:
io.logger_setup()
model = models.Cellpose(gpu=True,model_type='nuclei') # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, _, _, _ = model.eval(gauss, diameter=150, channels=channels, z_axis=0,stitch_threshold=0.1,flow_threshold=0.4,cellprob_threshold=0.0, min_size=100)

creating new log file
2025-06-20 12:38:49,808 [INFO] WRITING LOG OUTPUT TO C:\Users\kristin.gallik\.cellpose\run.log
2025-06-20 12:38:49,808 [INFO] 
cellpose version: 	3.1.1.2 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.7.0+cu118
2025-06-20 12:38:49,808 [INFO] ** TORCH CUDA version installed and working. **
2025-06-20 12:38:49,817 [INFO] >>>> using GPU (CUDA)
2025-06-20 12:38:49,819 [INFO] >> nuclei << model set to be used
2025-06-20 12:38:49,921 [INFO] >>>> loading model C:\Users\kristin.gallik\.cellpose\models\nucleitorch_0
2025-06-20 12:38:50,050 [INFO] >>>> model diam_mean =  17.000 (ROIs rescaled to this size during training)
2025-06-20 12:38:50,054 [INFO] channels set to [[0, 0]]
2025-06-20 12:38:50,055 [INFO] ~~~ FINDING MASKS ~~~
2025-06-20 12:38:50,055 [INFO] multi-stack tiff read in as having 27 planes 1 channels
2025-06-20 12:38:50,485 [INFO] 100%|##########| 4/4 [00:00<00:00, 40.00it/s]
2025-06-20 12:38:50,690 [INFO] network run in 0.30s
2025-06-20

100%|██████████| 26/26 [00:00<00:00, 152.95it/s]


2025-06-20 12:39:09,788 [INFO] masks created in 19.10s
2025-06-20 12:39:11,973 [INFO] >>>> TOTAL TIME 21.92 sec


In [149]:
voxel = [0.25,0.13,0.13]
viewer = napari.view_image(gauss, scale=voxel)
viewer.add_labels(masks,scale=voxel)

<Labels layer 'masks' at 0x16e5464b700>